<a href="https://colab.research.google.com/github/NavdeepTU/Deep-Learning/blob/main/02.%20ML%20basics%20with%20Keras/07.%20Save%20and%20load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyyaml h5py # Required to save models in HDF5 format

In [2]:
import os
import tensorflow as tf

In [3]:
# to demonstrate how to save and load weights, we'll use the MNIST dataset.
# To speed up these runs, use the first 1000 examples.

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
# Define a model

# define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(512, activation = 'relu', input_shape = (784,)),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
  ])

  model.compile(optimizer = 'adam',
                loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407050 (1.55 MB)
Trainable params: 407050 (1.55 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# Save checkpoints during training

# You can use a trained model without having to retrain it,
# or pick-up training where you left off in case the training process was interrupted.

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

# Train the model with new callback
model.fit(train_images,
          train_labels,
          epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback]) # Pass callback to training

Epoch 1/10
31/32 [============================>.] - ETA: 0s - loss: 1.1313 - sparse_categorical_accuracy: 0.6754
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 19ms/step - loss: 1.1269 - sparse_categorical_accuracy: 0.6770 - val_loss: 0.7021 - val_sparse_categorical_accuracy: 0.7890
Epoch 2/10
26/32 [=======================>......] - ETA: 0s - loss: 0.4358 - sparse_categorical_accuracy: 0.8762
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.4209 - sparse_categorical_accuracy: 0.8840 - val_loss: 0.5185 - val_sparse_categorical_accuracy: 0.8360
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2775 - sparse_categorical_accuracy: 0.9310
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 11ms/step - loss: 0.2775 - sparse_categorical_accuracy: 0.9310 - val_loss: 0.4615 - val_sparse_categorical_accuracy: 0.8560
Epoch 4/10
32/32 [=======

In [6]:
os.listdir(checkpoint_dir)

['cp.ckpt.data-00000-of-00001', 'cp.ckpt.index', 'checkpoint']

In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Untrained model accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 2.3870 - sparse_categorical_accuracy: 0.0970 - 198ms/epoch - 6ms/step
Untrained model accuracy:  9.70%


In [8]:
# Load the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.3945 - sparse_categorical_accuracy: 0.8700 - 89ms/epoch - 3ms/step
Restored model, accuracy: 87.00%


In [9]:
# Checkpoint callback options
# Train a new model, and save uniquely named checkpoints once every five epochs

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)

# create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq = 5*n_batches
)

# create a new model instance
model = create_model()

# save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch = 0))

# train the model with new callback
model.fit(train_images,
          train_labels,
          epochs = 50,
          batch_size = batch_size,
          callbacks = [cp_callback],
          validation_data = (test_images, test_labels),
          verbose = 0)


Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt


In [10]:
# Now, review the resulting checkpoints and choose the latest one
os.listdir(checkpoint_dir)

['cp-0020.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0035.ckpt.index',
 'cp-0025.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0040.ckpt.index',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0030.ckpt.index',
 'cp-0050.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'checkpoint']

In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [12]:
# test, reset the model, and load the latest checkpoint

# create a new model instance
model = create_model()

# load the previously saved weights
model.load_weights(latest)

# re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

32/32 - 0s - loss: 0.4831 - sparse_categorical_accuracy: 0.8770 - 228ms/epoch - 7ms/step
Restored model, accuracy: 87.70%
